In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json,re
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum,get_min_max
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline




df_train=pd.read_csv('dataset/origen.csv')
#take a 10% sample out of train
""" df_val =  df_train.sample(frac=0.1, random_state=42)
#drop the sample from train
df_train.drop(df_val.index, inplace=True)

X_val = df_val.drop(["averageRating",'Unnamed: 0'], axis=1) """
X = df_train.drop(["averageRating",'Unnamed: 0'], axis=1)
y = df_train["averageRating"]


In [2]:
X['runtimeMinutes'].replace(0, np.nan, inplace=True)
X['budget'].replace(0, np.nan, inplace=True)
X['revenue'].replace(0, np.nan, inplace=True)

print(X.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 879787 entries, 0 to 977540
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   numVotes              879787 non-null  int64  
 1   titleType             879787 non-null  object 
 2   isAdult               879787 non-null  float64
 3   startYear             879787 non-null  int64  
 4   endYear               879787 non-null  int64  
 5   runtimeMinutes        647118 non-null  float64
 6   genres_x              879785 non-null  object 
 7   directors             879787 non-null  object 
 8   writers               879787 non-null  object 
 9   seasonNumber          394234 non-null  float64
 10  episodeNumber         394234 non-null  float64
 11  ordering              333665 non-null  float64
 12  language              333665 non-null  object 
 13  attributes            333665 non-null  object 
 14  isOriginalTitle       333665 non-null  float64
 15  

In [ ]:

categorical_columns_with_nan = [
    c for c in X.columns if X[c].dtype == 'O' and X[c].isnull().any()]
categorical_columns_without_nan = [
    c for c in X.columns if X[c].dtype == 'O' and X[c].notnull().all()]

discrete_columns_with_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].isnull().any()]
discrete_columns_without_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].notnull().all()]


numerical_columns_with_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].isnull().any()]
numerical_columns_without_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].notnull().all()]

In [ ]:
from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, MeanMedianImputer
from sklearn.preprocessing import FunctionTransformer
def interpolate_numericals(df, group, category, method='linear'):
    # Fill in missing values using interpolation
    df[category].replace(0, np.nan, inplace=True)
    df[category].interpolate(method=method, inplace=True)
    
    # Store the fitted values in a dictionary
    if group not in interpolate_numericals.fitted_values:
        interpolate_numericals.fitted_values[group] = {}
    interpolate_numericals.fitted_values[group][category] = df[category].copy()
    
    return df

# Initialize the fitted values dictionary
interpolate_numericals.fitted_values = {}

pipeline_preprocessing = Pipeline([('numericalMissingIndicator',AddMissingIndicator(variables=numerical_columns_with_nan)),
                                   ('runtimeTransform',FunctionTransformer(lambda x: interpolate_numericals(x,'titleType','runtimeMinutes'))),
                                   ('budgetTransform',FunctionTransformer(lambda x: interpolate_numericals(x,'titleType','budget'))),
                                   ('revenueTransform',FunctionTransformer(lambda x: interpolate_numericals(x,'titleType','revenue'))),
                                   ])
#run the pipeline
pipeline_preprocessing.fit(X)
#Run and print the pipeline
print(pipeline_preprocessing.transform(X))

pipeline = Pipeline([('preprocessing',pipeline_preprocessing)])

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from joblib import Parallel, delayed

class GetExpDict(BaseEstimator, TransformerMixin):
    def __init__(self, group:list, categories:list):
        if type(group) == str:
            group = [group]
        if type(categories) == str:
            categories = [categories]
        self.group = group
        self.categories = categories
    
    def fit(self, X:pd.DataFrame, y=None):
        
        
        grupos = []
        for group in self.group:
            for category in self.categories:
                categorias = X[category].unique().tolist()
                categorias = [x.split(',') if type(x) == str else [] for x in categorias]
                categorias = [item for sublist in categorias for item in sublist]
                categorias = list(set(categorias))
                grupo = X.groupby(group)[category].apply(list).to_dict()
                if '0' in grupo:
                    del grupo['0']
                grupos.append((group,grupo,categorias))
        self.diccionarios=[]
        for group,grupo,categorias in grupos:
            
            dict_limpio = dict()
            for sub_grupo in grupo.keys():
                shows = grupo[sub_grupo]

                for dir in sub_grupo.split(','):
                    if dir in dict_limpio:
                        dict_limpio[dir] += shows
                    else:
                        dict_limpio[dir] = shows
            for sub_grupo in dict_limpio.keys():
                #split all the genres into a list
                dict_limpio[sub_grupo] = [x.split(',') if type(x) == str else [] for x in dict_limpio[sub_grupo]]
                #flatten the list
                dict_limpio[sub_grupo] = [item for sublist in dict_limpio[sub_grupo] for item in sublist]
                #turn the list into a dict and count the number of times each genre appears
                dict_limpio[sub_grupo] = dict((x,dict_limpio[sub_grupo].count(x)) for x in set(dict_limpio[sub_grupo]))
                # if not all genres are present, add the missing ones and set the count to 0
                for cat in categorias:
                    if cat not in dict_limpio[sub_grupo]:
                        dict_limpio[sub_grupo][cat] = 0
            self.diccionarios.append((group,dict_limpio,categorias))
        
        return self
    
    def transform(self, X):

        results = Parallel(n_jobs=-1)(delayed(sum_into_column)(x, diccionario, category) for group, diccionario, categorias in self.diccionarios for category in categorias for _, x in X.iterrows() if group == self.group)
        for i, (group, diccionario, categorias) in enumerate(self.diccionarios):
            if group == self.group:
                for j, category in enumerate(categorias):
                    X[f'{group}_exp_{category}'] = results[i*len(categorias)+j::len(categorias)]
        return X


pipeline_exp = Pipeline([
    ('set_exp_dict_genres', GetExpDict(group=['directors'], categories=['genres_x'])),
    # add other steps here
])

pipeline_exp.fit(X)


Pipeline(steps=[('set_exp_dict_genres',
                 GetExpDict(categories=['genres_x', 'titleType'],
                            group=['directors']))])

In [10]:
rrrr= pipeline_exp.transform(X)

In [ ]:
print(interpolate_numericals.fitted_values)